In [1]:
#installing necessary package
def get_necessay_package_installed():
  #function to install necessary package
  !pip install -q -U trl transformers accelerate
  !pip install -q datasets bitsandbytes einops scipy
  print("<<<<Necessary Package Installed>>>>")
get_necessay_package_installed()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.9 MB/s eta 0:00:00
<<<<Necessary Package Installed>>>>

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

In [4]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [7]:
def get_zeroshot_inference():
  text='''[INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Can you summarize the following text  :
  Question answering (QA) systems are a type of natural language processing (NLP) technology that provide
  precise and concise answers to questions posed in natural language. These systems have the potential to
  revolutionize the way we access information and can be applied in a wide range of fields including education,
  customer service, and health care.There are several approaches to building QA systems, including rule-based,
  information retrieval, and machine learning-based approaches. Rule-based systems rely on predefined rules and
  patterns to extract answers from a given text, while information retrieval systems use search algorithms to
  retrieve relevant information from a large database. Machine learning-based systems, on the other hand, use
  training data to learn to extract answers from text.One of the main challenges faced by QA systems is the need
  to understand the context and intent behind a question. This requires the system to have a deep understanding of
  the language and the ability to make inferences based on the given information. Another challenge is the need to
  extract relevant information from a large and potentially unstructured dataset.Despite these challenges, QA
  systems have a wide range of applications, including education, customer service, and health care.
  .[/INST]'''
  device = "cuda:0"

  inputs = tokenizer(text, return_tensors="pt").to(device)
  outputs = model.generate(**inputs, max_new_tokens=400)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [8]:
get_zeroshot_inference()

[INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Can you summarize the following text  :
  Question answering (QA) systems are a type of natural language processing (NLP) technology that provide
  precise and concise answers to questions posed in natural language. These systems have the potential to
  revolutionize the way we access information and can be applied in a wide range of fields including education,
  customer service, and health care.There are several approaches to building QA systems, including rule-based,
  information retrieval, and machine learning-based approaches. Rule-based systems rely on predefined rules and
  patterns to extract answers from a given text, while information retrieval systems use search algorithms to
  retrieve relevant information from a large database. Machine learning-based systems, on the other hand, use
  training data to learn to extract answers from text.One of the main ch

In [15]:
def get_oneshot_inference():
  text='''[INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Can you summarize the following text  :
  Context: Question answering (QA) systems are a type of natural language processing (NLP) technology that provide
  precise and concise answers to questions posed in natural language. These systems have the potential to
  revolutionize the way we access information and can be applied in a wide range of fields including education,
  customer service, and health care.There are several approaches to building QA systems, including rule-based,
  information retrieval, and machine learning-based approaches. Rule-based systems rely on predefined rules and
  patterns to extract answers from a given text, while information retrieval systems use search algorithms to
  retrieve relevant information from a large database. Machine learning-based systems, on the other hand, use
  training data to learn to extract answers from text.One of the main challenges faced by QA systems is the need
  to understand the context and intent behind a question. This requires the system to have a deep understanding of
  the language and the ability to make inferences based on the given information. Another challenge is the need to
  extract relevant information from a large and potentially unstructured dataset.Despite these challenges, QA
  systems have a wide range of applications, including education, customer service, and health care.

  summary: Question answering (QA) systems are a type of natural language processing (NLP) technology that provide precise and concise answers to questions posed in natural language. rule-based,
  information retrieval, and machine learning-based are different approaches in building QA systems.One of the main challenges faced by QA systems is the need
  to understand the context and intent behind a question. Another challenge is the need toextract relevant information from a large and potentially unstructured dataset. QA
  systems have a wide range of applications, including education, customer service, and health care. By understanding the current state of development and the potential impact of QA systems, we can better utilize these technologies to
  improve various industries and enhance the way we access information.[/INST]
  [INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Can you summarize the following text
  Context : Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations since before recorded history. It is named after the Roman god Jupiter.[19] When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows,[20] and is on average the third-brightest natural object in the night sky after the Moon and Venus.
  Jupiter is primarily composed of hydrogen with a quarter of its mass being helium, though helium comprises only about a tenth of the number of molecules. It may also have a rocky core of heavier elements,[21] but like the other giant planets, Jupiter lacks a well-defined solid surface. Because of its rapid rotation, the planet's shape is that of an oblate spheroid (it has a slight but noticeable bulge around the equator)
  Summary :
  .[/INST]'''
  device = "cuda:0"

  inputs = tokenizer(text, return_tensors="pt").to(device)
  outputs = model.generate(**inputs, max_new_tokens=400)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [16]:
get_oneshot_inference()

[INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Can you summarize the following text  :
  Context: Question answering (QA) systems are a type of natural language processing (NLP) technology that provide
  precise and concise answers to questions posed in natural language. These systems have the potential to
  revolutionize the way we access information and can be applied in a wide range of fields including education,
  customer service, and health care.There are several approaches to building QA systems, including rule-based,
  information retrieval, and machine learning-based approaches. Rule-based systems rely on predefined rules and
  patterns to extract answers from a given text, while information retrieval systems use search algorithms to
  retrieve relevant information from a large database. Machine learning-based systems, on the other hand, use
  training data to learn to extract answers from text.One of th

In [47]:
def get_few_shot_inference():
  text='''[INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Context : Context: NLP Cloud was founded in 2021 when the team realized there was no easy way to reliably leverage Natural Language Processing in production.
  Question: When was NLP Cloud founded?
  Answer: 2021 [/INST]
  [INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Context: NLP Cloud developed their API by mid-2020 and they added many pre-trained open-source models since then.
  Question: What did NLP Cloud develop?
  Answer: API [/INST]
  [INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Context: All plans can be stopped anytime. You only pay for the time you used the service. In case of a downgrade, you will get a discount on your next invoice.
  Question: When can plans be stopped?
  Answer: Anytime [/INST]
  [INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Context: The main challenge with GPT-J is memory consumption. Using a GPU plan is recommended.
  Question: Which plan is recommended for GPT-J?
  Answer: GPU plan [/INST]
  [INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Context:The main advantage with GPT-J is less pricing. .
  Question:What is the main advantage for GPT-J?
  Answer:
  .[/INST]'''
  device = "cuda:0"

  inputs = tokenizer(text, return_tensors="pt").to(device)
  outputs = model.generate(**inputs, min_length=1,max_new_tokens=1000)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [48]:
get_few_shot_inference()

[INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Context : Context: NLP Cloud was founded in 2021 when the team realized there was no easy way to reliably leverage Natural Language Processing in production.
  Question: When was NLP Cloud founded?
  Answer: 2021 [/INST]
  [INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Context: NLP Cloud developed their API by mid-2020 and they added many pre-trained open-source models since then.
  Question: What did NLP Cloud develop?
  Answer: API [/INST]
  [INST]<<SYS>>
  You are a helpful  assistant that provides information based on the given query in context.
  <</SYS>>
  Context: All plans can be stopped anytime. You only pay for the time you used the service. In case of a downgrade, you will get a discount on your next invoice.
  Question: When can plans be stopped?
  Answer: Anytime [/INST]
  [INST]<<